# Assignment 3_OPM 662
## 1. Formulate and solve the optimization problem
### 1, c & d)

In [1]:
import pandas as pd
import numpy as np
import docplex
from docplex.mp.model import Model 

#### Problem Instance 1

##### Create the model instance

In [2]:
model_P1 = Model("JettyScheduling_P1")

##### Define all sets

In [3]:
data = pd.ExcelFile("inst1.xlsx")
ships = list(data.parse("Problem_1", index_col = 0, header = None, usecols = "G", nrows = 8, skiprows = 2).index) #All the ships
full_cat = data.parse("Problem_1", index_col = 0, header = 0, usecols = "G:H", nrows = 8, skiprows = 1) #The categories of each ship
sta_unstrip = list(data.parse("Problem_1", index_col = None, header = 0, usecols = "B:E", nrows = 1, skiprows = 1).columns) #All the stations
sta = list(map(int, [i.replace("P", "") for i in sta_unstrip])) #All the stations' numbers
arr = data.parse("Problem_1", index_col = 0, header = 0, usecols = "J:K", nrows = 8, skiprows = 1) #Earliest arrival times of ship i
unload = data.parse("Problem_1", index_col = None, header = None, usecols = "B:E", nrows = 5, skiprows = 2) #Unloading time for ship category c at station alpha
big_M = 10000

In [21]:
print(ships)
print(full_cat)
print(sta)
print(arr)
print(unload)
print(sta_unstrip)

['Natalie', 'Neptun', 'Duandra', 'Waldhof', 'Panega', 'Megara', 'Onassis', 'Hansa', 'Carina', 'Atlantic']
          category
Natalie          2
Neptun           2
Duandra          3
Waldhof          4
Panega           1
Megara           5
Onassis          5
Hansa            2
Carina           4
Atlantic         3
[1, 2, 3, 4]
          arrival
Natalie         1
Neptun          3
Duandra         2
Waldhof         7
Panega          7
Megara          7
Onassis         3
Hansa          11
Carina          5
Atlantic        4
      1  2     3     4
0     3  5     7  9999
1  9999  6     7     9
2     4  3     7     2
3     5  7  9999     1
4     2  5     9     7
['P1', 'P2', 'P3', 'P4']


##### Define the decision variables

In [5]:
X = model_P1.continuous_var_matrix(ships, sta, name = "start_time") #The starting time of ship i entering station alpha
Y = model_P1.binary_var_dict(((i, j, alpha) for i in ships for j in ships for alpha in sta), name = "order") #1 if ship i is directly or indirectly in front of ship j in station alpha
W = model_P1.binary_var_matrix(ships, sta, name = "unload_station") #1 if ship i unloads at station alpha
p = model_P1.continuous_var_matrix(ships, sta, name = "process_time") #Processing time for ship i in station alpha

##### Define all constraints

In [6]:
model_P1.add_constraints(X[i,1] >= arr.loc[i,"arrival"] for i in ships) #The earliest start time into station 1 is not earlier than the earliest arrival time
model_P1.add_constraints(X[i,alpha+1] >= X[i,alpha]+p[i,alpha] for i in ships for alpha in range(1, len(sta))) #The earliest start time for ship i to enter the next station is only after ship i finishes processing in the previous station
model_P1.add_constraints(X[j,alpha] >= X[i,alpha]+p[i,alpha]+(Y[i,j,alpha]-1)*big_M for i in ships for j in ships if j!=i for alpha in sta) #Ship j can only start after ship i finish processing in station i if it is behind ship i, otherwise ship j can start at any time (under other constraints) in station i
model_P1.add_constraints(X[j,alpha] >= X[i,alpha]+X[i,alpha+1]-X[i,alpha]+(Y[i,j,alpha]-1)*big_M for i in ships for j in ships if j!=i for alpha in range(1, len(sta))) #Another constraints for ship j to be only able to start after ship i finish processing in station i if it is behind ship i, otherwise ship j can start at any time (under other constraints) in station i 
model_P1.add_constraints(Y[i,j,alpha]+Y[j,i,alpha] == 1 for i in ships for j in ships if j!=i for alpha in sta) #Ensure that either ship i is in front of ship j or ship j is in front of ship i
model_P1.add_constraints(Y[i,j,alpha] == Y[i,j,alpha+1] for i in ships for j in ships if j!=i for alpha in range(1, len(sta))) #If ship i is in front ship j in station alpha, ship i should also be in front of ship j in other stations
model_P1.add_constraints(model_P1.sum(W[i,alpha] for alpha in sta) == 1 for i in ships) #Each ship i can only unload in one station
model_P1.add_constraints(p[i,alpha] == 1+W[i,alpha]*(unload.loc[full_cat.loc[i,"category"]-1,alpha]-1) for i in ships for alpha in sta)#Processing time in the station is either 1 (when pass by) or the unloading time

[docplex.mp.LinearConstraint[](process_time_Exxon_1,EQ,unload_station_Exxon_1+1),
 docplex.mp.LinearConstraint[](process_time_Exxon_2,EQ,4unload_station_Exxon_2+1),
 docplex.mp.LinearConstraint[](process_time_Exxon_3,EQ,8unload_station_Exxon_3+1),
 docplex.mp.LinearConstraint[](process_time_Exxon_4,EQ,6unload_station_Exxon_4+1),
 docplex.mp.LinearConstraint[](process_time_Nina_1,EQ,2unload_station_Nina_1+1),
 docplex.mp.LinearConstraint[](process_time_Nina_2,EQ,4unload_station_Nina_2+1),
 docplex.mp.LinearConstraint[](process_time_Nina_3,EQ,6unload_station_Nina_3+1),
 docplex.mp.LinearConstraint[](process_time_Nina_4,EQ,9998unload_station_Nina_4+1),
 docplex.mp.LinearConstraint[](process_time_Moers_1,EQ,3unload_station_Moers_1+1),
 docplex.mp.LinearConstraint[](process_time_Moers_2,EQ,2unload_station_Moers_2+1),
 docplex.mp.LinearConstraint[](process_time_Moers_3,EQ,6unload_station_Moers_3+1),
 docplex.mp.LinearConstraint[](process_time_Moers_4,EQ,unload_station_Moers_4+1),
 docplex.mp

##### Define linear expression

In [7]:
J = model_P1.sum(X[i,4]+p[i,4]-arr.loc[i,"arrival"] for i in ships)

##### Define the objective function

In [8]:
model_P1.minimize(J)

##### Solve the model

In [9]:
model_P1.solve()

docplex.mp.solution.SolveSolution(obj=64,values={start_time_Exxon_1:2,st..

In [10]:
model_P1.solve_details
model_P1.report()
model_P1.print_information()
model_P1.print_solution()

* model JettyScheduling_P1 solved with objective = 64.000
Model: JettyScheduling_P1
 - number of variables: 352
   - binary=288, integer=0, continuous=64
 - number of constraints: 856
   - linear=856
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
objective: 64.000
  start_time_Exxon_1=2.000
  start_time_Exxon_2=4.000
  start_time_Exxon_3=5.000
  start_time_Exxon_4=6.000
  start_time_Nina_1=10.000
  start_time_Nina_2=13.000
  start_time_Nina_3=14.000
  start_time_Nina_4=15.000
  start_time_Moers_1=5.000
  start_time_Moers_2=6.000
  start_time_Moers_3=7.000
  start_time_Moers_4=8.000
  start_time_Tornado_1=4.000
  start_time_Tornado_2=5.000
  start_time_Tornado_3=6.000
  start_time_Tornado_4=7.000
  start_time_Kabul_1=1.000
  start_time_Kabul_2=2.000
  start_time_Kabul_3=3.000
  start_time_Kabul_4=4.000
  start_time_Clausthal_1=13.000
  start_time_Clausthal_2=14.000
  start_time_Clausthal_3=20.000
  start_time_Clausthal_4=21.000
  start_time_Opladen_1=6.000
  sta

##### Store the results

In [11]:
headers_unload_P1 = ["index","name", "order", "earliest arrival time", "start time", "unload at station"]
result_unload_order_P1 = pd.DataFrame(columns = headers_unload_P1)
a = 1
for i in ships:
    result_unload_order_P1.loc[len(result_unload_order_P1), "index"] = a
    a += 1
    result_unload_order_P1.loc[len(result_unload_order_P1)-1,"name"] = i
    result_unload_order_P1.loc[len(result_unload_order_P1)-1,"earliest arrival time"] = arr.loc[i,"arrival"]
    result_unload_order_P1.loc[len(result_unload_order_P1)-1,"start time"] = round(X[i,1].solution_value)
    for alpha in sta:
        if W[i,alpha].solution_value != 0.0:
            result_unload_order_P1.loc[len(result_unload_order_P1)-1,"unload at station"] = W[i,alpha].solution_value*alpha
for i in ships:
    result_unload_order_P1["order"] = result_unload_order_P1["start time"].rank() 
result_unload_order_P1 = result_unload_order_P1.set_index("index").rename_axis("ship i")

Sep_lists_P1 = []
for i in range(len(sta)):
    Sep_lists_P1.append(["" for j in range(21)])
for i in ships:
    for s in range(1, len(sta)):
        for t in range(round(X[i, s].solution_value), round(X[i, s+1].solution_value)):
            Sep_lists_P1[s-1][t-1] = i
    for t in range(round(X[i, 4].solution_value), round(X[i, 4].solution_value + p[i, 4].solution_value)):
        Sep_lists_P1[3][t-1] = i
rows_P1 = ["alpha = "+str(i) for i in sta]
final_result_P1 = pd.DataFrame(Sep_lists_P1, index = rows_P1, columns = [j for j in range(1,22)]).rename_axis("station | time")

In [12]:
print(final_result_P1)
print(result_unload_order_P1)

                   1      2      3        4        5        6        7   \
station | time                                                            
alpha = 1       Kabul  Exxon  Exxon  Tornado    Moers  Opladen  Opladen   
alpha = 2              Kabul           Exxon  Tornado    Moers            
alpha = 3                     Kabul             Exxon  Tornado    Moers   
alpha = 4                              Kabul             Exxon  Tornado   

                     8        9        10  ...     12         13         14  \
station | time                             ...                                
alpha = 1         Herta    Herta     Nina  ...   Nina  Clausthal              
alpha = 2       Opladen             Herta  ...              Nina  Clausthal   
alpha = 3                Opladen           ...                         Nina   
alpha = 4         Moers    Moers  Opladen  ...  Herta                         

                       15         16         17         18         19  \
s

#### Problem Instance 2

##### Create the model instance

In [13]:
model_P2 = Model("JettyScheduling_P2")

##### Define all sets

In [14]:
data = pd.ExcelFile("inst2.xlsx")
ships = list(data.parse("Problem_2", index_col = 0, header = None, usecols = "G", nrows = 10, skiprows = 2).index) #All the ships
full_cat = data.parse("Problem_2", index_col = 0, header = 0, usecols = "G:H", nrows = 10, skiprows = 1) #The categories of each ship
sta_unstrip = list(data.parse("Problem_2", index_col = None, header = 0, usecols = "B:E", nrows = 1, skiprows = 1).columns) #All the stations
sta = list(map(int, [i.replace("P", "") for i in sta_unstrip])) #All the stations' numbers
arr = data.parse("Problem_2", index_col = 0, header = 0, usecols = "J:K", nrows = 10, skiprows = 1) #Earliest arrival times of ship i
unload = data.parse("Problem_2", index_col = None, header = None, usecols = "B:E", nrows = 5, skiprows = 2) #Unloading time for ship category c at station alpha
big_M = 10000

In [15]:
print(ships)
print(full_cat)
print(sta)
print(arr)
print(unload)

['Natalie', 'Neptun', 'Duandra', 'Waldhof', 'Panega', 'Megara', 'Onassis', 'Hansa', 'Carina', 'Atlantic']
          category
Natalie          2
Neptun           2
Duandra          3
Waldhof          4
Panega           1
Megara           5
Onassis          5
Hansa            2
Carina           4
Atlantic         3
[1, 2, 3, 4]
          arrival
Natalie         1
Neptun          3
Duandra         2
Waldhof         7
Panega          7
Megara          7
Onassis         3
Hansa          11
Carina          5
Atlantic        4
      1  2     3     4
0     3  5     7  9999
1  9999  6     7     9
2     4  3     7     2
3     5  7  9999     1
4     2  5     9     7


##### Define the decision variables

In [16]:
X = model_P2.continuous_var_matrix(ships, sta, name = "start_time") #The starting time of ship i entering station alpha
Y = model_P2.binary_var_dict(((i, j, alpha) for i in ships for j in ships for alpha in sta), name = "order") #1 if ship i is directly or indirectly in front of ship j in station alpha
W = model_P2.binary_var_matrix(ships, sta, name = "unload_station") #1 if ship i unloads at station alpha
p = model_P2.continuous_var_matrix(ships, sta, name = "process_time") #Processing time for ship i in station alpha

##### Define all constraints

In [17]:
model_P2.add_constraints(X[i,1] >= arr.loc[i,"arrival"] for i in ships) #The earliest start time into station 1 is not earlier than the earliest arrival time
model_P2.add_constraints(X[i,alpha+1] >= X[i,alpha]+p[i,alpha] for i in ships for alpha in range(1, len(sta))) #The earliest start time for ship i to enter the next station is only after ship i finishes processing in the previous station
model_P2.add_constraints(X[j,alpha] >= X[i,alpha]+p[i,alpha]+(Y[i,j,alpha]-1)*big_M for i in ships for j in ships if j!=i for alpha in sta) #Ship j can only start after ship i finish processing in station i if it is behind ship i, otherwise ship j can start at any time (under other constraints) in station i
model_P2.add_constraints(X[j,alpha] >= X[i,alpha]+X[i,alpha+1]-X[i,alpha]+(Y[i,j,alpha]-1)*big_M for i in ships for j in ships if j!=i for alpha in range(1, len(sta))) #Another constraints for ship j to be only able to start after ship i finish processing in station i if it is behind ship i, otherwise ship j can start at any time (under other constraints) in station i 
model_P2.add_constraints(Y[i,j,alpha]+Y[j,i,alpha] == 1 for i in ships for j in ships if j!=i for alpha in sta) #Ensure that either ship i is in front of ship j or ship j is in front of ship i
model_P2.add_constraints(Y[i,j,alpha] == Y[i,j,alpha+1] for i in ships for j in ships if j!=i for alpha in range(1, len(sta))) #If ship i is in front ship j in station alpha, ship i should also be in front of ship j in other stations
model_P2.add_constraints(model_P2.sum(W[i,alpha] for alpha in sta) == 1 for i in ships) #Each ship i can only unload in one station
model_P2.add_constraints(p[i,alpha] == 1+W[i,alpha]*(unload.loc[full_cat.loc[i,"category"]-1,alpha]-1) for i in ships for alpha in sta)#Processing time in the station is either 1 (when pass by) or the unloading time

[docplex.mp.LinearConstraint[](process_time_Natalie_1,EQ,9998unload_station_Natalie_1+1),
 docplex.mp.LinearConstraint[](process_time_Natalie_2,EQ,5unload_station_Natalie_2+1),
 docplex.mp.LinearConstraint[](process_time_Natalie_3,EQ,6unload_station_Natalie_3+1),
 docplex.mp.LinearConstraint[](process_time_Natalie_4,EQ,8unload_station_Natalie_4+1),
 docplex.mp.LinearConstraint[](process_time_Neptun_1,EQ,9998unload_station_Neptun_1+1),
 docplex.mp.LinearConstraint[](process_time_Neptun_2,EQ,5unload_station_Neptun_2+1),
 docplex.mp.LinearConstraint[](process_time_Neptun_3,EQ,6unload_station_Neptun_3+1),
 docplex.mp.LinearConstraint[](process_time_Neptun_4,EQ,8unload_station_Neptun_4+1),
 docplex.mp.LinearConstraint[](process_time_Duandra_1,EQ,3unload_station_Duandra_1+1),
 docplex.mp.LinearConstraint[](process_time_Duandra_2,EQ,2unload_station_Duandra_2+1),
 docplex.mp.LinearConstraint[](process_time_Duandra_3,EQ,6unload_station_Duandra_3+1),
 docplex.mp.LinearConstraint[](process_time_D

##### Define linear expression

In [18]:
J = model_P2.sum(X[i,4]+p[i,4]-arr.loc[i,"arrival"] for i in ships)

##### Define the objective function

In [19]:
model_P2.minimize(J)

##### Solve the model

In [20]:
model_P2.solve()

DOcplexLimitsExceeded: **** Promotional version. Problem size limits (1000 vars, 1000 consts) exceeded, model has 520 vars, 1350 consts, CPLEX code=1016

In [ ]:
model_P2.solve_details
model_P2.report()
model_P2.print_information()
model_P2.print_solution()

##### Store the results

In [ ]:
headers_unload_P2 = ["index","name", "order", "earliest arrival time", "start time", "unload at station"]
result_unload_order_P2 = pd.DataFrame(columns = headers_unload_P2)
a = 1
for i in ships:
    result_unload_order_P2.loc[len(result_unload_order_P2), "index"] = a
    a += 1
    result_unload_order_P2.loc[len(result_unload_order_P2)-1,"name"] = i
    result_unload_order_P2.loc[len(result_unload_order_P2)-1,"earliest arrival time"] = arr.loc[i,"arrival"]
    result_unload_order_P2.loc[len(result_unload_order_P2)-1,"start time"] = round(X[i,1].solution_value)
    for alpha in sta:
        if W[i,alpha].solution_value != 0:
            result_unload_order_P2.loc[len(result_unload_order_P2)-1,"unload at station"] = W[i,alpha].solution_value*alpha
for i in ships:
    result_unload_order_P2["order"] = result_unload_order_P2["start time"].rank() 
result_unload_order_P2 = result_unload_order_P2.set_index("index").rename_axis("ship i")

Sep_lists_P2 = []
for i in range(len(sta)):
    Sep_lists_P2.append(["" for j in range(27)])
for i in ships:
    for s in range(1, len(sta)):
        for t in range(round(X[i, s].solution_value), round(X[i, s+1].solution_value)):
            Sep_lists_P2[s-1][t-1] = i
    for t in range(round(X[i, 4].solution_value), round(X[i, 4].solution_value + p[i, 4].solution_value)):
        Sep_lists_P2[3][t-1] = i
rows_P2 = ["alpha = "+str(i) for i in sta]
final_result_P2 = pd.DataFrame(Sep_lists_P2, index = rows_P2, columns = [j for j in range(1,28)]).rename_axis("station | time")

In [ ]:
print(final_result_P2)
print(result_unload_order_P2)

#### Save the results

In [ ]:
with pd.ExcelWriter("Jetty_Scheuling.xlsx") as writer:
    result_unload_order_P1.to_excel(writer, sheet_name = "Problem 1_Order")
    final_result_P1.to_excel(writer, sheet_name = "Problem 1_Final Result")
    result_unload_order_P2.to_excel(writer, sheet_name = "Problem 2_Order")
    final_result_P2.to_excel(writer, sheet_name = "Problem 2_Final Result")